# Using Machine Learning Tools Assignment 1

## Overview

In this assignment, you will apply some popular machine learning techniques to the problem of predicting bike rental demand. A data set has been provided containing records of bike rentals in Seoul, collected during 2017-18.

## General instructions

This assignment is divided into several tasks. Use the spaces provided in this notebook to answer the questions posed in each task. Note that some questions require writing a small amount of code and some require graphical results. It is your responsibility to make sure your responses are clearly labelled and your code has been fully executed (with the correct results displayed) before submission!

**Do not** manually edit the data set file we have provided! For marking purposes, it's important that your code is written to run correctly on the original data file.

When creating graphical output, label is clearly, with appropriate titles, xlabels and ylabels, as appropriate.

Chapter 2 of the textbook is based on a similar workflow to this assignment, so you may look there for some further background and ideas. You can also use any other general resources on the internet that are relevant although do not use ones which directly relate to these questions with this dataset (which would normally only be found in someone else's assignment answers). If you take a large portion of code or text from the internet then you should reference where this was taken from, but we do not expect any references for small pieces of code. Taking, and adapting, small portions of code is expected and is common practice when solving real problems.

## The following code imports some of the essential libraries that you will need. You should not need to modify it, but you are expected to import other libraries as needed.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

**STEP01:** 
Load the data set from the csv file (SeoulBikeData.csv) into a DataFrame, and summarise it with the pandas functions `describe()` and `info()`.

Download the data set from MyUni using the link provided on the assignment page. A paper that describes one related version of this dataset is: Sathishkumar V E, Jangwoo Park, and Yongyun Cho. 'Using data mining techniques for bike sharing demand prediction in metropolitan city.' Computer Communications, Vol.153, pp.353-366, March, 2020. Feel free to look at this if you want more information about the dataset.

The data is stored in a CSV (comma separated variable) file and contains the following information 

 - Date: year-month-day
 - Rented Bike Count: Count of bikes rented at each hour
 - Hour: Hour of the day
 - Temperature: Temperature in Celsius
 - Humidity: %
 - Windspeed: m/s
 - Visibility: 10m
 - Dew point temperature: Celsius
 - Solar radiation: MJ/m2
 - Rainfall: mm
 - Snowfall: cm
 - Seasons: Winter, Spring, Summer, Autumn
 - Holiday: Holiday/No holiday
 - Functional Day: NoFunc(Non Functional Hours), Fun(Functional hours)

**Load the data set from the csv file into a DataFrame, and summarise it with at least two appropriate pandas functions.**

_Points:_ 2

In [ ]:
data = pd.read_csv("SeoulBikeData.csv")

data.info()
data.describe()
data.head()

# The following code is used by the autograder, don't change it
step1_sol = data.shape

grader.check("step01")

**STEP02:** To get a feeling for the data it is a good idea to do some form of simple visualisation. Display a set of histograms for the features as they are right now, prior to any cleaning steps.

_Points:_ 0

In [ ]:
# Create 4x3 subplots
fig, axes = plt.subplots(4, 4, figsize=(15, 12))  

# Plot histograms for all columns 
for i, col in enumerate(data.columns):
    sns.histplot(data[col], ax=axes[i//4, i%4], kde=True)
    axes[i//4, i%4].set_title(col)
    axes[i//4, i%4].set_xlabel('')

fig.suptitle('Distribution of Bike Renting Factors', fontsize=14)
plt.tight_layout()
plt.show()


grader.check("step02")

**STEP03:** The "Functioning Day" feature records whether the bike rental was open for business on that day. For this assignment we are only interested in predicting demand on days when the business is open, so remove rows from the DataFrame where the business is closed. After doing this, delete the Functioning Day feature from the DataFrame and verify that this worked. **While you might solve this question in one way, be aware that there are multiple methods to achieve the same result in pandas, and these methods might be the subject of a question in the PracQuiz1 about this assignment.**

_Points:_ 3

In [ ]:
data = data.drop(data[data['Functioning Day'] != "Yes" ].index)
data = data.drop('Functioning Day', axis=1)

# The following code is used by the autograder
# make sure your variable that contains that data from this step is the one assigned to step3_data
step3_data = data.copy()

grader.check("step03")

**STEP04:** 
The goal is to predict bike rental demand using historical data. To achieve this, you will use regression techniques with "Bike Rental Count" as the target feature for this prediction, *but for this*, it is important that all other features in the data are numerical. Two of the features in the data, "Holiday" and "Season", need to be converted to numerical format. Write code to convert the "Holiday" feature to 0 or 1 from its current format. For the "Season" feature, add 4 new columns, labeled as "Winter", "Spring", "Summer", and "Autumn" then remove the "Season" column. Each of these columns should store a 0 or 1, depending on the corresponding season in each row. **While you might solve this question in one way, be aware that there are multiple methods to achieve the same result in pandas, and these methods might be the subject of a question in the PracQuiz1 about this assignment.**

_Points:_ 3

In [ ]:

# Map Holiday state to 1 and No Holiday to 0
data['Holiday'] = data['Holiday'].map({'Holiday': 1, 'No Holiday': 0})

# Convert elements in Seasons to dummies variable
seasons = pd.get_dummies(data["Seasons"], prefix=None, drop_first=False, dtype=int)  
data.drop("Seasons", axis=1, inplace=True)  
data = pd.concat([data, seasons], axis=1)


# The following code is used by the autograder, don't change it
step4_data = data.copy()

grader.check("step04")

**STEP05**: It is known that bike rentals depend strongly on whether it's a weekday or a weekend. Replace the Date feature with a Weekday feature that stores 0 or 1 depending on whether the date represents a weekend or weekday. **While you might solve this question in one way, be aware that there are multiple methods to achieve the same result in pandas, and these methods might be the subject of a question in the PracQuiz1 about this assignment.**

_Points:_ 3

In [ ]:
# convert from object to pandas date format
data["Date"]= pd.to_datetime(data["Date"],format= "%d/%m/%Y")

# find day of the week from the date
data["Date"] = data["Date"].dt.day_of_week

# define condition for weekday
def weekday_def(day):
    if day >= 5:
        return 0
    else:
        return 1
    
# map conditions across columns
data["Date"] = data["Date"].map(weekday_def)

# rename columns
data.rename(columns={"Date":"Weekday"}, inplace=True)


# The following code is used by the autograder
# make sure your variable that contains that data from this step is the one assigned to step5_data
step5_data = data.copy()

grader.check("step05")

**STEP06** Convert all the remaining data to numerical format, with any non-numerical entries set to NaN.

_Points:_ 3

In [ ]:
# convert rainfall and snowfall to numeric
data["Rainfall(mm)"] = pd.to_numeric(data["Rainfall(mm)"], errors='coerce')
data["Snowfall (cm)"] = pd.to_numeric(data["Snowfall (cm)"], errors='coerce')

# The following code is used by the autograder
# make sure your variable that contains that DataFrame from this step is the one assigned to step6_data
step6_data = data.copy()

grader.check("step06")

**STEP07** Examin the data and identify problematic entries. Set any problematic values in the numerical data to `np.nan` and check that this has worked. Once this is done, specify a **sklearn *pipeline* that will perform imputation** to replace problematic entries (nan values) with an appropriate **median** value ***and* any other pre-processing** that you think should be used. Just specify the pipeline - do ***not*** run it now.

_Points:_ 3

In [ ]:
# set negative values from Humidity and Visibility to nan
data["Humidity (%)"] = data["Humidity (%)"].apply(lambda x: np.nan if x < 0 else x)
data["Visibility (10m)"] = data["Visibility (10m)"].apply(lambda x: np.nan if x < 0 else x)

# keep the variable name pipeline_step7 as you will use it in STEP09
pipeline_step7 = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
    ('scaler', StandardScaler())  # Standardize features
])

# The following code is used by the autograder
# make sure your variable that contains the solutions from this step is the one assigned to step7_data as follows
step7_data = [data.copy(),pipeline_step7]

grader.check("step07")

**STEP08:** Generate a pre-processed version of the entire dataset by applying the pipeline defined in STEP07. Then, calculate the correlation of each feature with the target using either the pandas function corr() or numpy corrcoef() and find the 3 attributes that are the most correlated with bike rentals. 

_Points:_ 3

In [ ]:
preprocessed_data = pipeline_step7.fit_transform(data)

correlation = data.corr()

# correlation heat map
plt.figure(figsize=(15,10))
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.2f', linewidths=2)
plt.show()

# top_3 should be an array of 3 strings ['attribute name', 'attribute name','attribute name']
top_3 = ['Temperature (C)', 'Winter', 'Hour']
print(top_3)
# The following code is used by the autograder
# make sure your variable that contains a list of the names of the top 3 attributes is assigned to step3_data
step8_data = top_3.copy()

grader.check("step08")

**STEP09:** Divide the data into training and test sets using where 20% of the data is kept for testing. Create a pipeline that includes the linear regression model in addition to the pipeline defined in STEP07. Fit the pipeline to the training set and calculate the `rmse` of the fit to evaluate its performance. As a comparison, compute the `rmse` that would be obtained by predicting the mean value of bike rentals for all training examples.

_Points:_ 3

In [ ]:
# you can use make_pipline to create a new pipiline by adding a model at the end of pipeline_step7 or you can simply create a new pipeline. 
# whatever you end up doing, make sure it is called pipeline_step9
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

pipeline_step9 = make_pipeline(pipeline_step7, LinearRegression())

y_train, y_test = train_test_split(data['Rented Bike Count'], test_size=0.2, random_state=42)
X_train, x_test = train_test_split(data.drop('Rented Bike Count', axis=1), test_size=0.2, random_state=42)


# calculate the RMSE of the fit to the training data
result = pipeline_step9.fit(X_train, y_train)
rmse_train = mean_squared_error(y_train, result.predict(X_train), squared=False)
# calculate the RMSE of the baseline model (by predicting the mean value of bike rentals for all training examples)
baseline_pred = y_train.mean()
rmse_baseline = mean_squared_error(y_train, [baseline_pred]*len(y_train), squared=False)

print("RMSE for training data:", rmse_train)
print("RMSE for baseline (predicting mean):", rmse_baseline)

# The following code is used by the autograder
# make sure your variable that contains that data from this step is the one assigned to step3_data
step9_data = [rmse_train,rmse_baseline]

In [ ]:
#The following code will show a visualisation of the fit for your linear regression.
# I will use your pipeline_step9 to predict on 200 points from the training data
subset_size = 200
y_train_pred = pipeline_step9.predict(X_train[:subset_size])

# Then I create a scatterplot of predicted vs actual values using your variables from the cell above
ax = sns.scatterplot(x=y_train[:subset_size], y=y_train_pred)
# A perfect solution would look like the red line
sns.lineplot(x=y_train[:subset_size], y=y_train[:subset_size], color='red')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')

grader.check("step09")

**STEP10:**  Fit a Kernel Ridge regression model (imported from sklearn.kernel_ridge) to the X_train data from STEP09. Build a new pipeline that includes the Kernel Ridge regression model in addition to the pipeline defined in STEP07, and fit it to the training data using default settings. Generate a scatter plot of the predicted values against the actual values for the training data, and calculate the RMSE of the fit to the training data.

_Points:_ 3

In [ ]:
from sklearn.kernel_ridge import KernelRidge

# you can use make_pipline to create a new pipiline by adding a model at the end of pipeline_step7 or you can simply create a new pipeline. 
# whatever you end up doing, make sure it is called pipeline_step10
pipeline_step10 = make_pipeline(pipeline_step7, KernelRidge())


# make predictions on the training data
y_pred_train_KR = pipeline_step10.fit(X_train, y_train).predict(X_train)

# calculate rmse for training data
rmse_train_KR = mean_squared_error(y_train, y_pred_train_KR, squared=False)
print('Kernel Ridge model RMSE on training data:', rmse_train_KR)

# The following code is used by the autograder
# make sure your variable that contains that data from this step is the one assigned to step3_data
step10_data = [rmse_train_KR,pipeline_step10]

In [ ]:
# I will use your pipeline_step10 to predict on 200 points from the training data
subset_size = 200
y_train_pred = pipeline_step10.predict(X_train[:subset_size])

# Then create a scatterplot of predicted vs actual values using your variables from the cell above
ax = sns.scatterplot(x=y_train[:subset_size], y=y_train_pred)
sns.lineplot(x=y_train[:subset_size], y=y_train[:subset_size], color='red')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')

grader.check("step10")

**STEP11:** Fit a Support Vector Regression (from sklearn.svm import SVR). As you did for STEP10, create a new pipeline using the pipelinr from STEP07 and this model and fit it to your training data, using the default settings. Again, calculate the RMSE of the fit to the training data.

_Points:_ 4

In [ ]:
from sklearn.svm import SVR

# you can use make_pipline to create a new pipiline by adding a model at the end of pipeline_step7 or you can simply create a new pipeline. 
# whatever you end up doing, make sure it is called pipeline_step11
pipeline_step11 = make_pipeline(pipeline_step7, SVR())

# make predictions on the training data
y_pred_train_SVR = pipeline_step11.fit(X_train, y_train).predict(X_train)

# calculate rmse for training data
rmse_train_SVR = mean_squared_error(y_train, y_pred_train_SVR, squared=False)
print('Support Vector Regression model RMSE on training data:', rmse_train_SVR)


# The following code is used by the autograder
# make sure your variable that contains that data from this step is the one assigned to step3_data
step11_data = [rmse_train_SVR,pipeline_step11]

In [ ]:
# use your pipeline_step10 to predict on 200 points from the training data
subset_size = 200
y_train_pred = pipeline_step11.predict(X_train[:subset_size])

# Then create a scatterplot of predicted vs actual values using your variables from the cell above
ax = sns.scatterplot(x=y_train[:subset_size], y=y_train_pred)
sns.lineplot(x=y_train[:subset_size], y=y_train[:subset_size], color='red')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')

grader.check("step11")

**STEP12:** Perform a 10 fold cross validation for each of the three model (LinearRegression,KernelRidge,SVR). This splits the training set (that we've used above) into 10 equal size subsets, and uses each in turn as the validation set while training a model with the other 9. You should therefore have 10 rmse values for each cross validation run. Find the mean and standard deviation of the rmse values obtained for each model for the validation splits.

_Points:_ 5

In [ ]:
# you might need some or all of the following imports
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score


kfold = KFold(n_splits=10, shuffle=True, random_state=42)

#Linear Regression CV mean and std RMSE from the 10 folds:
rmse_LR = []
for train_index, test_index in kfold.split(data):
    X_train, X_test = data.drop('Rented Bike Count', axis=1).iloc[train_index], data.drop('Rented Bike Count', axis=1).iloc[test_index]
    y_train, y_test = data['Rented Bike Count'].iloc[train_index], data['Rented Bike Count'].iloc[test_index]
    result = pipeline_step9.fit(X_train, y_train)
    rmse = mean_squared_error(y_test, result.predict(X_test), squared=False)
    rmse_LR.append(rmse)
    
rmse_LR_mean = np.mean(rmse_LR)
rmse_LR_std  = np.std(rmse_LR)
print('Linear Regression CV Scores:') 
print(f'Mean: {rmse_LR_mean:.2f}, Std: {rmse_LR_std:.2f}\n')


# Kernel Ridge Regression CV mean and std:
rmse_KR = []
for train_index, test_index in kfold.split(data):
    X_train, X_test = data.drop('Rented Bike Count', axis=1).iloc[train_index], data.drop('Rented Bike Count', axis=1).iloc[test_index]
    y_train, y_test = data['Rented Bike Count'].iloc[train_index], data['Rented Bike Count'].iloc[test_index]
    result = pipeline_step10.fit(X_train, y_train)
    rmse = mean_squared_error(y_test, result.predict(X_test), squared=False)
    rmse_KR.append(rmse)
rmse_KR_mean = np.mean(rmse_KR)
rmse_KR_std  = np.std(rmse_KR)
print('Kernel Ridge Regression CV Scores:') 
print(f'Mean: {rmse_KR_mean:.2f}, Std: {rmse_KR_std:.2f}\n')


# Support Vector Regression CV mean and std:
rmse_SVR = []
for train_index, test_index in kfold.split(data):
    X_train, X_test = data.drop('Rented Bike Count', axis=1).iloc[train_index], data.drop('Rented Bike Count', axis=1).iloc[test_index]
    y_train, y_test = data['Rented Bike Count'].iloc[train_index], data['Rented Bike Count'].iloc[test_index]
    result = pipeline_step11.fit(X_train, y_train)
    rmse = mean_squared_error(y_test, result.predict(X_test), squared=False)
    rmse_SVR.append(rmse)
rmse_SVR_mean = np.mean(rmse_SVR)
rmse_SVR_std  = np.std(rmse_SVR)
print('Support Vector Regression CV Scores:') 
print(f'Mean: {rmse_SVR_mean:.2f}, Std: {rmse_SVR_std:.2f}\n')


# The following code is used by the autograder
step12_data = [rmse_LR_mean,rmse_KR_mean,rmse_SVR_mean]

grader.check("step12")

**STEP13:** Both the Kernel Ridge Regression and Support Vector Regression have hyperparameters that can be adjusted to suit the problem. Use grid search to systematically compare the generalisation performance (rmse) obtained with different hyperparameter settings (still with 10-fold CV). Use the sklearn function GridSearchCV to do this.

For KernelRidge, vary the hyperparameter alpha. (note, if you are using KernelRidge as the last step in a pipeline, alpha is refered to as kernelridge__alpha) 

For SVR, vary the hyperparameter C. (note, if you are using SVR as the last step in a pipeline, C is refered to as SVR__C)

Find the hyperparameter setting for each medel.

Finally, train and apply both models, with the best hyperparameter settings, to the test set and report the performance as rmse.

_Points:_ 5

In [47]:
from sklearn.model_selection import GridSearchCV

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data.drop('Rented Bike Count', axis=1), data['Rented Bike Count'], test_size=0.2, random_state=42)

# Define the GridSearchCV objects for each model
kr_param = {'kernelridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100,1000]}
kr_cv = GridSearchCV(pipeline_step10, param_grid=kr_param, cv=kfold, scoring='neg_root_mean_squared_error')
svr_param = {'svr__C': [0.001, 0.01, 0.1, 1, 10, 100,1000]}
svr_cv = GridSearchCV(pipeline_step11, param_grid=svr_param, cv=kfold, scoring='neg_root_mean_squared_error')

# Fit the GridSearchCV objects to the training data
kr_cv.fit(X_train, y_train)
svr_cv.fit(X_train, y_train)

# Print the best hyperparameter setting for each model
print("Best hyperparameter setting for Kernel Ridge Regression:", kr_cv.best_params_)
print("Best hyperparameter setting for Support Vector Regression:", svr_cv.best_params_)

# Create pipeline using the best hyperparameter
pipeline_best_kr = kr_cv.best_estimator_
pipeline_best_svr = svr_cv.best_estimator_

# Train and apply the chosen model to the test set
kr_predictions = pipeline_best_kr.fit(X_train, y_train).predict(x_test)
kr_rmse = mean_squared_error(y_test, kr_predictions, squared=False)

svr_predictions = pipeline_best_svr.fit(X_train, y_train).predict(x_test)
svr_rmse = mean_squared_error(y_test, svr_predictions, squared=False)

print("Kernel Ridge Regression RMSE on test set:", kr_rmse)
print("Support Vector Regression RMSE on test set:", svr_rmse)

# The following code is used by the autograder
# make sure your variable that contains that data from this step is the one assigned to step3_data
step13_data = [kr_rmse , svr_rmse, kr_cv , svr_cv,pipeline_best_kr,pipeline_best_svr]

KeyboardInterrupt: 

In [ ]:
# use your pipeline_best_svr to predict on 200 points from the training data
subset_size = 200
y_train_pred = pipeline_best_svr.predict(X_train[:subset_size])

# Then create a scatterplot of predicted vs actual values using your variables from the cell above
ax = sns.scatterplot(x=y_train[:subset_size], y=y_train_pred)
sns.lineplot(x=y_train[:subset_size], y=y_train[:subset_size], color='red')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')

grader.check("step13")